In [1]:
import azureml.core

from azureml.core import Experiment, Workspace, Dataset, Datastore
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

StatementMeta(, , , SessionStarting, )

In [ ]:
subscription_id = "f80606e5-788f-4dc3-a9ea-2eb9a7836082"
resource_group = "rg-synapse-training"
workspace_name = "mlworkspace-training"
experiment_name = "satraining-nyc_taxi-20210527095441"

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
experiment = Experiment(ws, experiment_name)

StatementMeta(, , , Waiting, )

In [ ]:
df = spark.sql("SELECT * FROM default.nyc_taxi")

datastore = Datastore.get_default(ws)
dataset = TabularDatasetFactory.register_spark_dataframe(df, datastore, name = experiment_name + "-dataset")

StatementMeta(, , , Waiting, )

In [ ]:
automl_config = AutoMLConfig(spark_context = sc,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "tipped",
                             primary_metric = "accuracy",
                             experiment_timeout_hours = 1,
                             max_concurrent_iterations = 4,
                             enable_onnx_compatible_models = False)

StatementMeta(, , , Waiting, )

In [ ]:
run = experiment.submit(automl_config)

StatementMeta(, , , Waiting, )

In [ ]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Machine Learning portal: {}</a>".format(run.get_portal_url(), run.id))

StatementMeta(, , , Waiting, )

In [ ]:
run.wait_for_completion()

import mlflow

# Get best model from automl run
best_run, non_onnx_model = run.get_output()

artifact_path = experiment_name + "_artifact"

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    # Save the model to the outputs directory for capture
    mlflow.sklearn.log_model(non_onnx_model, artifact_path)

    # Register the model to AML model registry
    mlflow.register_model("runs:/" + run.info.run_id + "/" + artifact_path, "satraining-nyc_taxi-20210527095441-Best")

StatementMeta(, , , Waiting, )